# Packages

In [1]:
import os
import numpy as np
import pandas as pd
import pickle
import helper as hp

In [2]:
state = 'ny'
# state = 'ca'

data_dir = f'../Data_{state}/'
results_dir = f'../Results_{state}/'
fig_dir = f'../Figures_{state}/'

# Resstock

In [3]:
# Read resstock building characteristics data
res_building_state = pd.read_csv(f'../Data_{state}/res_building_{state}_meta.csv')

# Create a list of county IDs
county_id_df = res_building_state.loc[:, ['in.county', 'in.resstock_county_id']]
county_id_df = county_id_df.drop_duplicates()
county_id_list = county_id_df['in.county'].to_list()
county_id_list.sort()

# Get lists of unique hvac types
res_heating_type_list = list(pd.unique(res_building_state['in.hvac_heating_type']))
res_cooling_type_list = list(pd.unique(res_building_state['in.hvac_cooling_type']))


In [4]:
# Import ResStock data
resstock_amy2018_state_dir = os.path.join(data_dir, f'resstock_amy2018/{state}')

# Read correction factors for resstock in state
res_corr =  pd.read_csv(os.path.join(resstock_amy2018_state_dir, f'correction_factors_{state}_2018.csv'))
res_corr = pd.concat([res_corr, res_corr.tail(1)], axis=0)
date_list = pd.date_range('2018-01-01', '2019-01-01', freq='1D')
res_corr = res_corr.set_index(date_list)
res_corr = res_corr.drop(columns=['Month', 'day_of_month', 'build_existing_model.state'])
res_corr = res_corr.resample('1H').ffill()
res_corr = res_corr.drop(res_corr.tail(1).index)

In [5]:
resstock_amy2018_bldg_dir = f'../Data_{state}/resstock_amy2018/building/'
resstock_amy2018_bldg_processed_dir = os.path.join(resstock_amy2018_bldg_dir, 'processed')
if not os.path.isdir(resstock_amy2018_bldg_processed_dir):
    os.makedirs(resstock_amy2018_bldg_processed_dir)

overwrite = True

for county_id in county_id_list:
    print('County ID', county_id)
    # If county's directory exists
    county_dir = os.path.join(resstock_amy2018_bldg_dir, county_id)
    if os.path.isdir(county_dir):
        
        # If processed pickle file doesn't already exist, process the data
        filename = os.path.join(resstock_amy2018_bldg_processed_dir, f'{county_id}_heat.pkl')
        if not os.path.isfile(filename) or overwrite:
            # Loop through hvac heating types
            res_bldg_heating_tot_dict = dict()
            for heating_type in res_heating_type_list:
                cond = (res_building_state['in.county']==county_id) & (res_building_state['in.hvac_heating_type']==heating_type)
                res_bldg_id_sublist = list(res_building_state[cond]['bldg_id'])
                res_bldg_wt_sublist = list(res_building_state[cond]['weight'])
                print(heating_type, ':', len(res_bldg_id_sublist))
                if len(res_bldg_id_sublist) > 0:
                    res_bldg_heating_tot_dict[heating_type] = hp.calc_bldg_ts_tot(res_bldg_id_sublist, res_bldg_wt_sublist, 
                                                                                    county_dir, res_corr)
                else:
                    res_bldg_heating_tot_dict[heating_type] = None
            # Write to pickle
            pickle.dump(res_bldg_heating_tot_dict, open(filename, 'wb') )
            print('Finished processing data and saved in:', filename)
        else:
            print('Processed file already exists:', filename)

        # If processed pickle file doesn't already exist, process the data
        filename = os.path.join(resstock_amy2018_bldg_processed_dir, f'{county_id}_cool.pkl')
        if not os.path.isfile(filename) or overwrite:
            # Loop through hvac cooling types
            res_bldg_cooling_tot_dict = dict()
            for cooling_type in res_cooling_type_list:
                cond = (res_building_state['in.county']==county_id) & (res_building_state['in.hvac_cooling_type']==cooling_type)
                res_bldg_id_sublist = list(res_building_state[cond]['bldg_id'])
                res_bldg_wt_sublist = list(res_building_state[cond]['weight'])
                print(cooling_type, ':', len(res_bldg_id_sublist))
                if len(res_bldg_id_sublist) > 0:
                    res_bldg_cooling_tot_dict[cooling_type] = hp.calc_bldg_ts_tot(res_bldg_id_sublist, res_bldg_wt_sublist, 
                                                                                    county_dir, res_corr)
                else:
                    res_bldg_cooling_tot_dict[cooling_type] = None
            # Write to pickle
            pickle.dump(res_bldg_cooling_tot_dict, open(filename, 'wb'))
            print('Finished processing data and saved in:', filename)
        else:
            print('Processed file already exists:', filename)

County ID G3600010
Non-Ducted Heating : 200
Ducted Heating : 353
Ducted Heat Pump : 19
None : 0
Finished processing data and saved in: ../Data_ny/resstock_amy2018/building/processed\G3600010_heat.pkl
None : 99
Room AC : 189
Central AC : 265
Heat Pump : 19
Finished processing data and saved in: ../Data_ny/resstock_amy2018/building/processed\G3600010_cool.pkl
County ID G3600030
Non-Ducted Heating : 41
Ducted Heating : 66
Ducted Heat Pump : 0
None : 0
Finished processing data and saved in: ../Data_ny/resstock_amy2018/building/processed\G3600030_heat.pkl
None : 35
Room AC : 37
Central AC : 35
Heat Pump : 0
Finished processing data and saved in: ../Data_ny/resstock_amy2018/building/processed\G3600030_cool.pkl
County ID G3600050
Non-Ducted Heating : 992
Ducted Heating : 1062
Ducted Heat Pump : 68
None : 36
Finished processing data and saved in: ../Data_ny/resstock_amy2018/building/processed\G3600050_heat.pkl
None : 357
Room AC : 996
Central AC : 737
Heat Pump : 68
Finished processing data an